# Stochastic Nonparametric Envelopment of Data (StoNED)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

### Objective: Estimates a log-transformed cost function model and technicial inefficiency

In [9]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Prepare and read data

Download the Excel data from <https://github.com/ds2010/StoNED-Python/tree/master/Data>

Data description is now available at <https://github.com/ds2010/StoNED-Python/blob/master/Data/data_description.ipynb>

In [10]:
# import data
df = pd.read_excel ('data.xlsx')

In [11]:
# output (total cost)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

Note 1: Converting Numpy array to list by `array.tolist()` can speed up the computation.

In [12]:
# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

### Stage 1: Convex Nonparametric Least Squares estimation

In [13]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

crt   = "mult"
func  = "cost"
pps   = "crs"

model = CNLS.cnls(y, x, crt, func, pps)
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

Note 2:
     # crt     = "addi" : Additive composite error term
     #         = "mult" : Multiplicative composite error term
     # func    = "prod" : production frontier
     #         = "cost" : cost frontier
     # pps     = "vrs"  : variable returns to scale production possibility sets (pps)
     #         = "crs"  : constant returns to scale pps

In [14]:
# display estimates
model.e.display() 
model.b.display() 

e : residuals
    Size=89, Index=i
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :  None :    0.03591704716833715 :  None : False : False :  Reals
      1 :  None :    0.02603548450043448 :  None : False : False :  Reals
      2 :  None :       0.20219661941764 :  None : False : False :  Reals
      3 :  None : -0.0045179666020388224 :  None : False : False :  Reals
      4 :  None :    0.17430052203709456 :  None : False : False :  Reals
      5 :  None :     -0.119122488270739 :  None : False : False :  Reals
      6 :  None :  -0.011301337908518495 :  None : False : False :  Reals
      7 :  None :     0.1192172009953594 :  None : False : False :  Reals
      8 :  None :    0.37369843926301544 :  None : False : False :  Reals
      9 :  None :   -0.04018422500498858 :  None : False : False :  Reals
     10 :  None :    0.02303955229784499 :  None : False : False :  Reals
     11 :  None :    -0.0777953929783989 :  None : False : False :  Reals
   

In [15]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

### Stage 2: Residuals decomposition

In [16]:
# decomposition by method od moment (MoM)
method      = 'MoM'
TE          = StoNED.stoned(y, eps, func, method, crt)

Note 3:

     # func    = "prod": production frontier;  
     #         = "cost": cost frontier
     # method  = "MOM" : Method of moments
     #         = "QLE" : Quasi-likelihood estimation 
     #         = "NKD" : Nonparametric kernel deconvolution (TBA...)
     # crt     = "addi": Additive composite error term
     #         = "mult": Multiplicative composite error term

In [17]:
# display technicial inefficiency
TE

(array([0.1105059 , 0.10726824, 0.17919513, 0.09787208, 0.16596215,
        0.07021879, 0.09590972, 0.14170339, 0.26922118, 0.08804024,
        0.10630599, 0.07891384, 0.20199328, 0.20802471, 0.09204495,
        0.06548633, 0.17570468, 0.16364092, 0.09518586, 0.07434675,
        0.06871303, 0.04879982, 0.17192134, 0.07533884, 0.14266604,
        0.12058246, 0.19984476, 0.05795195, 0.13600177, 0.10781271,
        0.09874208, 0.03398479, 0.08677779, 0.18592391, 0.09801795,
        0.15939441, 0.05438541, 0.06407529, 0.0710183 , 0.08885777,
        0.08597888, 0.11361462, 0.12061187, 0.08263648, 0.07637351,
        0.0679864 , 0.10768497, 0.12768059, 0.06679231, 0.06765987,
        0.17623639, 0.09270356, 0.17832946, 0.17808135, 0.11284351,
        0.05410961, 0.05255803, 0.0765087 , 0.05803269, 0.10541287,
        0.05462534, 0.07833093, 0.08919577, 0.10393152, 0.26496472,
        0.07212552, 0.12661961, 0.08582709, 0.12697565, 0.0513405 ,
        0.0858708 , 0.12659847, 0.05765455, 0.07